In [1]:
import os
os.chdir("../")
%pwd

'd:\\projects\\pet\\MLFlow\\Feature_Eng'

In [2]:
import pandas as pd
data = pd.read_csv("D:/projects/pet/MLFlow/Feature_Eng/artifacts/data_ingestion/BTC.csv")



In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23981 entries, 0 to 23980
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    23981 non-null  object 
 1   open    23981 non-null  float64
 2   high    23981 non-null  float64
 3   low     23981 non-null  float64
 4   close   23981 non-null  float64
 5   volume  23981 non-null  float64
dtypes: float64(5), object(1)
memory usage: 1.1+ MB


In [4]:
data.isnull().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [6]:
from mlFeature.constants import *
from mlFeature.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )
        return data_validation_config
config = ConfigurationManager()
data_validation_config = config.get_data_validation_config()

[2023-09-19 20:30:19,846: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-19 20:30:19,849: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-19 20:30:19,852: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-19 20:30:19,854: INFO: common: created directory at: artifacts]
[2023-09-19 20:30:19,856: INFO: common: created directory at: artifacts/data_validation]


In [7]:
import os 
from mlFeature import logger 

class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

data_validation = DataValiadtion(config=data_validation_config)
data_validation.validate_all_columns()


True

In [ ]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e